<a href="https://colab.research.google.com/github/ghlim40/BT/blob/main/my_python_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#라이브러리 설치
!pip install pyupbit
!pip install datetime
!pip install numpy
!pip install pyjwt
!pip install datetime
!pip install schedule
!pip install Prophet

     |████████████████████████████████| 103 kB 8.4 MB/s 
     |████████████████████████████████| 60 kB 3.4 MB/s 
     |████████████████████████████████| 251 kB 10.9 MB/s 
     |████████████████████████████████| 65 kB 2.5 MB/s 
     |████████████████████████████████| 49 kB 5.6 MB/s 
     |████████████████████████████████| 43 kB 2.3 MB/s 
  Created wheel for Prophet: filename=prophet-1.0.1-py3-none-any.whl size=6639680 sha256=c9625ce365f58970f5ee36f71e6e548db7442df9d6fb3a07bfc2699239dc3cfa
  Stored in directory: /root/.cache/pip/wheels/4e/a0/1a/02c9ec9e3e9de6bdbb3d769d11992a6926889d71567d6b9b67
Successfully built Prophet
  Attempting uninstall: cmdstanpy
    Found existing installation: cmdstanpy 0.9.5
    Uninstalling cmdstanpy-0.9.5:
      Successfully uninstalled cmdstanpy-0.9.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fbprophet 0.7.1 requires cmdstanpy

In [ ]:
#라이브러리 임포트
import pyupbit
import datetime
import time, calendar
import numpy as np
import datetime
import schedule
from fbprophet import Prophet

In [ ]:
# set variables
coin = "KRW-BTC"   # 매매를 진행할 가상화폐의 ticker
fees = 0.0005      # 수수료

In [ ]:
#access key & secret key
access = "your-access"
secret = "your-secret"

In [ ]:
#목표 가격 측정
# -1은 오늘, -2는 어제데이터
def get_targetPrice(df, K) :
    range = df['high'][-2] - df['low'][-2] #전일 고가와 전일 저가의 차이값
    return df['open'][-1] + range * K #당일 시가 + range값에 상수k를 곱하여 더함

In [ ]:
#코인 구매기능
def buy_all(coin) :
    #get_balance : 보유중인 잔고 조회
    balance = upbit.get_balance("KRW") * 0.9995
    
    #업비트 최소 주문금액이 5000원 이상
    if balance >= 5000 : 
        print(upbit.buy_market_order(coin, balance))

In [ ]:
#코인 판매기능
def sell_all(coin) :
    balance = upbit.get_balance(coin)
    price = pyupbit.get_current_price(coin)
    
    #업비트 최소 주문금액이 5000원 이상
    if price * balance >= 5000 : 
        print(upbit.sell_market_order(coin, balance))


In [ ]:
#누적수익률 반환
# drr = Daily Rate Of Return 하루수익률
# crr = Cumulative Rate Of Return 누적수익률
# mdd = Max Draw Down, dd = Draw Down 손해율

def get_crr(df, fees, K) :
    df['range'] = df['high'].shift(1) - df['low'].shift(1)
    df['targetPrice'] = df['open'] + df['range'] * K
    df['drr'] = np.where(df['high'] > df['targetPrice'], (df['close'] / (1 + fees)) / (df['targetPrice'] * (1 + fees)) , 1)
    return df['drr'].cumprod()[-2]

In [ ]:
#해당 일자의 가장 적절한 K값 선정
def get_best_K(coin, fees) :
    df = pyupbit.get_ohlcv(coin, interval = "day", count = 21)
    max_crr = 0
    best_K = 0.5
    for k in np.arange(0.0, 1.0, 0.1) :
        crr = get_crr(df, fees, k)
        if crr > max_crr :
            max_crr = crr
            best_K = k
    return best_K

In [ ]:
#미래 가측 예측
predicted_close_price = 0
def predict_price(coin):
    """Prophet으로 당일 종가 가격 예측"""
    global predicted_close_price
    df = pyupbit.get_ohlcv(coin, interval="minute60")
    df = df.reset_index()
    df['ds'] = df['index']
    df['y'] = df['close']
    data = df[['ds','y']]
    model = Prophet()
    model.fit(data)
    future = model.make_future_dataframe(periods=24, freq='H')
    forecast = model.predict(future)
    closeDf = forecast[forecast['ds'] == forecast.iloc[-1]['ds'].replace(hour=9)]
    if len(closeDf) == 0:
        closeDf = forecast[forecast['ds'] == data.iloc[-1]['ds'].replace(hour=9)]
    closeValue = closeDf['yhat'].values[0]
    predicted_close_price = closeValue
predict_price(coin)
schedule.every().hour.do(lambda: predict_price(coin))

In [ ]:
#거래 전략
if __name__ == '__main__': 
    try:
        upbit = pyupbit.Upbit(access, secret)

        K = 0.5            # targetPrice 를 구할 때 사용할 K 값
    
        start_balance = upbit.get_balance("KRW")
        df = pyupbit.get_ohlcv(coin, count = 2, interval = "day")
        targetPrice = get_targetPrice(df, get_best_K(coin, fees))
        print(datetime.datetime.now().strftime('%y/%m/%d %H:%M:%S'), "\t\tBalance :", start_balance, "KRW \t\tYield :", ((start_balance / start_balance) - 1) * 100, "% \t\tNew targetPrice :", targetPrice, "KRW")

        while True :
            now = datetime.datetime.now()
            if now.hour == 9 and now.minute == 0 and now.second == 0:    # when am 09:02:00
                sell_all(coin)
                df = pyupbit.get_ohlcv(coin, count = 2, interval = "day")
                targetPrice = get_targetPrice(df, get_best_K(coin, fees))
                time.sleep(1)

                cur_balance = upbit.get_balance("KRW")
                print(now.strftime('%y/%m/%d %H:%M:%S'), "\t\tBalance :", cur_balance, "KRW \t\tYield :", ((cur_balance / start_balance) - 1) * 100, "% \t\tNew targetPrice :", targetPrice, "KRW")
            
            elif targetPrice <= pyupbit.get_current_price(coin) and pyupbit.get_current_price(coin) < predicted_close_price :
                buy_all(coin)
                start_time = df.index[-1]
                end_time = start_time + datetime.timedelta(days=1)
                if end_time > now :
                    print((end_time - now).seconds)
                    time.sleep((end_time - now).seconds - 60)
    
            time.sleep(1)

    except Exception as e:
        print(e)
        time.sleep(1)


401
잘못된 엑세스 키입니다.
invalid_access_key
InValidAccessKey
unsupported operand type(s) for /: 'NoneType' and 'NoneType'
